# Example of DOV search methods for lithologische beschrijvingen

@pjhaest
date 20180908

## Use cases:
* Select records in a bbox
* Select records in a bbox with selected properties
* Select records in a municipality
* Get records using info from wfs fields, not available in the standard output dataframe

In [1]:
%matplotlib inline
import os, sys
import inspect
import warnings; warnings.simplefilter('ignore')

In [2]:
# check pydov path
import pydov
print(pydov.__path__)

['c:\\_workingdir\\pydov\\pydov']


## Get information about code base

In [5]:
from pydov.search.interpretaties import LithologischeBeschrijvingenSearch
ip_litho = LithologischeBeschrijvingenSearch()

# information about the HydrogeologischeStratigrafie type (In Dutch):
print(ip_litho.get_description())


Wanneer een boring wordt uitgevoerd wordt er een beschrijving gemaakt van het materiaal dat men aangeboord heeft. Men beschrijft de textuur, het soort materiaal, de kleur, mineralen, fossielen, ... per diepte-interval. Dit kan zowel op het veld gebeuren als achteraf indien de stalen bewaard worden. Deze beschrijvingen kunnen gedaan worden door bijvoorbeeld de boormeester of door een geoloog. Meestal wordt de boring beschreven per diepte-interval waarover men hetzelfde materiaal herkent. De mate van detail van de beschrijving kan afhankelijk zijn van het doeleinde waarvoor men de boring uitvoert of wie de beschrijving doet: is het om een geologische kaart te maken, dienen er monsters genomen te worden voor laboratoriumanalyses, ... In DOV worden de beschrijvingen van ontsluitingen boven het aardoppervlak, aangeduid als boringen met diepte 0 m, beschreven van 0 m tot 0 m.


In [6]:
# information about the available fields for a HydrogeologischeStratigrafie object
fields = ip_litho.get_fields()
# print available fields
for f in fields.values():
    print(f['name'])

generated_id
Auteurs
Opdrachten
diepte_laag_van
diepte_laag_tot
Datum
Z_mTAW
Geldig_van
beschrijving
Proefnummer
pkey_boring
gemeente
betrouwbaarheid_interpretatie
x
y
pkey_interpretatie
Type_proef
Geldig_tot


In [7]:
# print information for a certain field
fields['beschrijving']

{'cost': 10,
 'definition': 'Lithologische beschrijving van de laag in vrije tekst (onbeperkt in lengte)',
 'name': 'beschrijving',
 'notnull': False,
 'type': 'string'}

The cost is an arbitrary attribute to indicate if the information is retrieved from a wfs query (cost = 1), 
or from an xml (cost = 10)

In [8]:
# if an attribute can have several values, these are listed under 'values', e.g. for 'Type_proef':
fields['Type_proef']

{'cost': 1,
 'definition': 'Het type van de proef waarvan de lithologische beschrijving een interpretatie is.',
 'name': 'Type_proef',
 'notnull': False,
 'type': 'string',
 'values': ['Boring', 'Sondering']}

## Try-out of use cases

### Select interpretations in a bbox

In [9]:
# Get all lithological descriptions in a bounding box (llx, lly, ulx, uly)
# the pkey_boring link is not available below, but is in the df
df = ip_litho.search(location=(152145, 204930, 153150, 206935))
df = df[df.beschrijving.notnull()]
print(df.head())

[000/016] cccccccccccccccc
                                  pkey_interpretatie  \
0  https://www.dov.vlaanderen.be/data/interpretat...   
1  https://www.dov.vlaanderen.be/data/interpretat...   
2  https://www.dov.vlaanderen.be/data/interpretat...   
3  https://www.dov.vlaanderen.be/data/interpretat...   
4  https://www.dov.vlaanderen.be/data/interpretat...   

                                         pkey_boring  \
0  https://www.dov.vlaanderen.be/data/boring/1973...   
1  https://www.dov.vlaanderen.be/data/boring/1973...   
2  https://www.dov.vlaanderen.be/data/boring/1973...   
3  https://www.dov.vlaanderen.be/data/boring/1973...   
4  https://www.dov.vlaanderen.be/data/boring/1973...   

  betrouwbaarheid_interpretatie         x         y  diepte_laag_van  \
0                          goed  152634.0  205540.0              0.0   
1                          goed  152634.0  205540.0              1.0   
2                          goed  152634.0  205540.0              2.0   
3          

### Select interpretations in a bbox with selected properties

In [10]:
# list available query methods
methods = [i for i,j in inspect.getmembers(sys.modules['owslib.fes'], 
                                           inspect.isclass) 
           if 'Property' in i]
print(methods)
from owslib.fes import PropertyIsGreaterThanOrEqualTo

['PropertyIsBetween', 'PropertyIsEqualTo', 'PropertyIsGreaterThan', 'PropertyIsGreaterThanOrEqualTo', 'PropertyIsLessThan', 'PropertyIsLessThanOrEqualTo', 'PropertyIsLike', 'PropertyIsNotEqualTo', 'PropertyIsNull', 'SortProperty']


The property feature methodes listed above are available from the owslib module. These were not adapted for use in pydov.

In [11]:
# Get deep boreholes in a bounding box 
from owslib.fes import PropertyIsEqualTo
# the propertyname can be any of the fields of the hydrogeological interpretations object that belong to the wfs source
# the literal is always a string, no matter what its definition is in the boring object (string, float...)
query = PropertyIsGreaterThanOrEqualTo(
        propertyname='betrouwbaarheid_interpretatie', literal='goed')
df = ip_litho.search(location=(153145, 206930, 153150, 206935),
                        query=query
                       )
print(df.head())

[000/001] c
                                  pkey_interpretatie  \
0  https://www.dov.vlaanderen.be/data/interpretat...   
1  https://www.dov.vlaanderen.be/data/interpretat...   
2  https://www.dov.vlaanderen.be/data/interpretat...   
3  https://www.dov.vlaanderen.be/data/interpretat...   
4  https://www.dov.vlaanderen.be/data/interpretat...   

                                         pkey_boring  \
0  https://www.dov.vlaanderen.be/data/boring/1974...   
1  https://www.dov.vlaanderen.be/data/boring/1974...   
2  https://www.dov.vlaanderen.be/data/boring/1974...   
3  https://www.dov.vlaanderen.be/data/boring/1974...   
4  https://www.dov.vlaanderen.be/data/boring/1974...   

  betrouwbaarheid_interpretatie         x         y  diepte_laag_van  \
0                          goed  153147.0  206931.0              0.0   
1                          goed  153147.0  206931.0              0.5   
2                          goed  153147.0  206931.0              1.0   
3                         

### Select interpretations in a municipality

In [12]:
query = PropertyIsEqualTo(propertyname='gemeente',
                          literal='Aartselaar')
df = ip_litho.search(query=query)
print(df.head())

[000/039] ccccccccccccccccccccccccccccccccccccccc
                                  pkey_interpretatie  \
0  https://www.dov.vlaanderen.be/data/interpretat...   
1  https://www.dov.vlaanderen.be/data/interpretat...   
2  https://www.dov.vlaanderen.be/data/interpretat...   
3  https://www.dov.vlaanderen.be/data/interpretat...   
4  https://www.dov.vlaanderen.be/data/interpretat...   

                                         pkey_boring  \
0  https://www.dov.vlaanderen.be/data/boring/1879...   
1  https://www.dov.vlaanderen.be/data/boring/1879...   
2  https://www.dov.vlaanderen.be/data/boring/1879...   
3  https://www.dov.vlaanderen.be/data/boring/1879...   
4  https://www.dov.vlaanderen.be/data/boring/1879...   

  betrouwbaarheid_interpretatie         x         y  diepte_laag_van  \
0                          goed  151929.0  202628.0              0.0   
1                          goed  151929.0  202628.0              0.2   
2                          goed  151929.0  202628.0         

### Visualize results

Using Folium, we can display the results of our search on a map.

In [13]:
# import the necessary modules (not included in the requirements of pydov!)
import folium
from folium.plugins import MarkerCluster
from pyproj import Proj, transform

In [14]:
# convert the coordinates to lat/lon for folium
def convert_latlon(x1, y1):
    inProj = Proj(init='epsg:31370')
    outProj = Proj(init='epsg:4326')
    x2,y2 = transform(inProj, outProj, x1, y1)
    return x2, y2
df['lon'], df['lat'] = zip(*map(convert_latlon, df['x'], df['y'])) 
# convert to list
loclist = df[['lat', 'lon']].values.tolist()

In [16]:
# initialize the Folium map on the centre of the selected locations, play with the zoom until ok
fmap = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=12)
marker_cluster = MarkerCluster().add_to(fmap)
for loc in range(0, len(loclist)):
    # limit marker size for folium (:10)
    folium.Marker(loclist[loc], popup=df['beschrijving'][loc][:10]).add_to(marker_cluster)
fmap